In [2]:
import numpy as np
import PIL
import platform

# Check versions
print("Python Version:", platform.python_version())
print("Installed Pillow Version:", PIL.__version__)
print("Installed Numpy Version:", np.__version__)

Python Version: 3.7.1
Installed Pillow Version: 8.4.0
Installed Numpy Version: 1.21.2


TODO:
* K-Means clustering for primary colors in image - I think sklearn K-means, gaussian, and dbscan should be interesting [documentation](https://scikit-learn.org/stable/modules/clustering.html).
* Ben Day dots in recursive fashion(?) - if color less than some acceptable spread, then dot. Otherwise, subdivide into kinda squares and move on

In [ ]:
# https://www.geeksforgeeks.org/how-to-convert-images-to-numpy-array/
# https://scikit-image.org/docs/stable/user_guide/getting_started.html